<a href="https://colab.research.google.com/github/dxiong2001/malibu-ml/blob/html_parser/html_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
#!pip install beautifulsoup4

In [107]:
from bs4 import BeautifulSoup
import requests
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re
import unicodedata

import spacy

nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [108]:
#url='https://www.npr.org/2022/07/14/1111618620/secret-service-erased-texts-from-two-day-period-spanning-jan-6-attack-watchdog-s'
#url = "https://www.popsci.com/environment/climate-change-wildlife-migration/"
url = "https://www.popsci.com/science/omicron-coronavirus-variant/"
page = requests.get(url)
#page.content

In [109]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())
#list(soup.children)
p = soup.find_all('p', class_='')
processed_text1 = [item.get_text() for item in p]

processed_text2=[]
for processed in processed_text1:
    
  p2 = processed.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'").replace('…', '...').replace('–', '-')
  
  processed_text2.append(unicodedata.normalize('NFKD', p2).encode('ascii', 'ignore').decode('utf-8'))

print(processed_text2)

['Preliminary clinical data from South Africa suggest that the COVID-19 Omicron variant may be less dangerous than initially feared. ', 'The South African Medical Research Council posted a report on Saturday outlining early Omicron cases from several hospitals. They found that patients infected with the Omicron strain tended to have less severe diseasemost of the hospitalized patients did not need supplemental oxygen, a few developed COVID pneumonia, and fewer still required intensive care. ', '"The signals are a bit encouraging" but Omicron\'s risk profile is still incomplete, Anthony Fauci, head of the National Institute of Allergy and Infectious Diseases, told CNN on Sunday. More clinical data from around the world will need to be assessed for epidemiologists to get a true sense of where Omicron stands in relation to other variants. "Clearly, in South Africa, Omicron has a transmission advantage," Fauci added, "although it\'s too early to make any definitive statements about it, thu

In [110]:
quotes = []

#for p in processed_text:
#  quotes.append(re.findall('"([^"]*)"', p))

sentences = [sent_tokenize(sent) for sent in processed_text2]

combined_sentences = []
for sent in sentences:
  combined_sentences += sent
# i=0
# for sent in sentences:
#   combined_sentence = ""
#   for s in sent:
#     combined_sentence += s
#   sentences[i] = combined_sentence
#   i=i+1



quotes_incomplete = [re.findall('"([^"]*)"', paragraph) for paragraph in processed_text2]
quotes_incomplete = [q for q in quotes_incomplete if q != []]
quotes_incomplete2 = [j for i in quotes_incomplete for j in i]

quotes_complete =[]

i=0
for comb in combined_sentences:
  
  if(quotes_incomplete2[i] in comb):
    if(comb not in quotes_complete):
      quotes_complete.append(comb)
    i = i + 1
    while i < len(quotes_incomplete2) and quotes_incomplete2[i] in comb:
        i = i + 1
    
  
  if(i==len(quotes_incomplete2)):
      break

print(quotes_complete)
#print(quotes)

['"The signals are a bit encouraging" but Omicron\'s risk profile is still incomplete, Anthony Fauci, head of the National Institute of Allergy and Infectious Diseases, told CNN on Sunday.', '"Clearly, in South Africa, Omicron has a transmission advantage," Fauci added, "although it\'s too early to make any definitive statements about it, thus far it does not look like there\'s a great degree of severity to it."', '"We\'re just not seeing the number of patients that have been seen in previous surges who are seriously ill, even this soon into the surge," Michael Osterholm, director of the University of Minnesota\'s Center for Infectious Diseases Research and Policy, told STAT, adding that he is "impressed by the relative lack of severe illness" with Omicron so far.', 'Data from the South African report show that the patients included tended to skew younger, which the report\'s authors write could be a vaccination effect, "as 57 percent of people over the age of 50 have been vaccinated i

In [111]:
##need to handle additional p tag text not relevant to article

joined_sentences = ' '.join(combined_sentences)
nlp_processed_text = nlp(joined_sentences)

named_entities=[]
for entities in nlp_processed_text.ents:
  named_entities.append((entities.text, entities.label_))

people = []
for n in named_entities:
  if(n[1]=="PERSON"):
    people.append(n)

print(people)

[('COVID-19 Omicron', 'PERSON'), ('Anthony Fauci', 'PERSON'), ('Michael Osterholm', 'PERSON'), ('Marion Koopmans', 'PERSON'), ('Venky Soundararajan', 'PERSON'), ('Nference', 'PERSON')]
